In [7]:
# --------------------------------------------
#                   TEST LCU
# --------------------------------------------
from src import lcu
lcu = lcu.LCU()

instr_list = ["NOP", "EXIT", "SADD R1, ZERO, LAST", "SADD R1, SRF(3), LAST", "SADDI R1, ONE, 7", "SSUB SRF(4), SRF(4), SRF(4)", "JUMP ONE, 7", "BGEPD ONE, ZERO, 5"]

for instr in instr_list:
    srf_read, srf_str, hex_word = lcu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + hex_word)

NOP: -1, -1, 0x0
EXIT: -1, -1, 0x1c00
SADD R1, ZERO, LAST: -1, -1, 0xb8340
SADD R1, SRF(3), LAST: 3, -1, 0xb0340
SADDI R1, ONE, 7: -1, -1, 0xfc347
SSUB SRF(4), SRF(4), SRF(4): 4, 4, 0x90400
JUMP ONE, 7: -1, -1, 0xfda07
BGEPD ONE, ZERO, 5: -1, -1, 0xdd605


In [8]:
# --------------------------------------------
#                   TEST LSU
# --------------------------------------------
from src import lsu
lsu = lsu.LSU()

instr_list = ["SADD R0, ONE, ONE/LD.VWR VWR_A", "SADD R0, SRF(5), ONE/SH.IL.UP", "SADD SRF(5), SRF(5), ONE/LD.VWR SRF"]

for instr in instr_list:
    srf_read, srf_str, hex_word = lsu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + hex_word)

SADD R0, ONE, ONE/LD.VWR VWR_A: -1, -1, 0x45538
SADD R0, SRF(5), ONE/SH.IL.UP: 5, -1, 0xc4538
SADD SRF(5), SRF(5), ONE/LD.VWR SRF: 5, 5, 0x5c530


In [9]:
# --------------------------------------------
#                   TEST RCs
# --------------------------------------------
from src import rc
rc = rc.RC()

instr_list = ["NOP", "SADD VWR_A, VWR_A, VWR_B", "SADD VWR_A, SRF(3), VWR_B", "LOR R0, RCB, MIN_INT", "MUL.FP R0, RCB, MIN_INT"]
# DIV.FP R0, RCB, MIN_INT") Error: float point div not supported
for instr in instr_list:
    srf_read, srf_str, vwr_str, hex_word = rc.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + hex_word)

NOP: -1, -1, 0x0
SADD VWR_A, VWR_A, VWR_B: -1, -1, 0x420
SADD VWR_A, SRF(3), VWR_B: 3, -1, 0xc420
LOR R0, RCB, MIN_INT: -1, -1, 0x1f522
MUL.FP R0, RCB, MIN_INT: -1, -1, 0x1f5a2


In [10]:
# --------------------------------------------
#                   TEST MXCU
# --------------------------------------------
from src import mxcu
mxcu = mxcu.MXCU()

instr_list = ["NOP", "SADD R1, ONE, LAST", "LOR R1, ONE, SRF(3)"]
srf_sel = -1
srf_we = 0
alu_srf_write = 0
vwr_row_we = [0,0,0,0]
vwr_sel = 0
for instr in instr_list:
    hex_word = mxcu.asmToHex(instr, srf_sel, srf_we, alu_srf_write, vwr_row_we, vwr_sel)
    print(instr + ": " + hex_word)

NOP: 0x0
SADD R1, ONE, LAST: 0x5699000
LOR R1, ONE, SRF(3): 0x54690c0


In [11]:
# --------------------------------------------
#                   COMPILE
# --------------------------------------------
from src import simulator
kernel_path='./kernels/test_asm/'
sim = simulator.SIMULATOR()
sim.compileAsm(kernel_path, version="_v1", nInstrPerCol=11, colUsage=[True, False], imem_start_addr=3)

Processing file: ./kernels/test_asm/instructions_v1.csv...


In [12]:
# Using two cols
#sim.compileAsm(kernel_path, version="_v2", nInstrPerCol=5, colUsage=[True, True], imem_start_addr=3)

In [13]:
# --------------------------------------------
#                   SIMULATE
# --------------------------------------------
from src.simulator import *
from random import randint

sim = SIMULATOR()

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/test_asm/' # Path
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 11 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF

sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)

# --------------------------------------------
#                LOAD SPM DATA
# --------------------------------------------
# Load vector by vector in the positions you want
random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
nline = 3
sim.setSPMLine(nline, random_vector)
# Or, load the entire content of the SPM
data = []
for nline in range(SPM_NLINES):
    random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
    data.append(random_vector)
sim.loadSPMData(data)

# --------------------------------------------
#               SIMULATE EXECUTION
# --------------------------------------------
show_lcu = ['R0']
show_srf = ['R3', 'R5']
show_lsu = []
show_rcs = [['R1'],['R0'],['OWN'],[]]
show_mxcu = ['R1']
display_ops = [show_lcu, show_lsu, show_mxcu, show_rcs, show_srf]

#sim.run(kernel_path, version="_v1", display_ops=display_ops)